## Benchmark Lorenz 96 linear and nonlinear filters

In this notebook, we are interested in the sequential inference 



References: 


[1] Evensen, G., 1994. Sequential data assimilation with a nonlinear quasi‐geostrophic model using Monte Carlo methods to forecast error statistics. Journal of Geophysical Research: Oceans, 99(C5), pp.10143-10162.

[2] Asch, M., Bocquet, M. and Nodet, M., 2016. Data assimilation: methods, algorithms, and applications. Society for Industrial and Applied Mathematics.

[3] Bishop, C.H., Etherton, B.J. and Majumdar, S.J., 2001. Adaptive sampling with the ensemble transform Kalman filter. Part I: Theoretical aspects. Monthly weather review, 129(3), pp.420-436. 

[4] Lorenz, E.N., 1963. Deterministic nonperiodic flow. Journal of atmospheric sciences, 20(2), pp.130-141.

[5] Spantini, A., Baptista, R. and Marzouk, Y., 2019. Coupling techniques for nonlinear ensemble filtering. arXiv preprint arXiv:1907.00389.

### The basic steps
To carry out sequential inference in `AdaptiveTransportMap`, we need to carry out a few basic steps:
* **Specify the problem**: Define the state-space model: initial condition, dynamical and observation models (including process and observation noise)
* **Specify the inflation parameters**: Determine the levels of covariance inflation to properly balance the dynamical system and the observations from the truth system
* **Specify the filter**: Choose the ensemble filter to assimilate the observations in the state estimate
* **Perform the sequential inference**: Perform the sequential inference

We will go through all of these here.

In [1]:
using Revise
using LinearAlgebra
using AdaptiveTransportMap
using Statistics
using Distributions
using OrdinaryDiffEq
using JLD

┌ Info: Precompiling AdaptiveTransportMap [bdf749b0-1400-4207-80d3-e689c0e3f03d]
└ @ Base loading.jl:1278
┌ Warning: Type annotations on keyword arguments not currently supported in recipes. Type information has been discarded
└ @ RecipesBase ~/.julia/packages/RecipesBase/92zOw/src/RecipesBase.jl:116
┌ Warning: Type annotations on keyword arguments not currently supported in recipes. Type information has been discarded
└ @ RecipesBase ~/.julia/packages/RecipesBase/92zOw/src/RecipesBase.jl:116
┌ Warning: Type annotations on keyword arguments not currently supported in recipes. Type information has been discarded
└ @ RecipesBase ~/.julia/packages/RecipesBase/92zOw/src/RecipesBase.jl:116
┌ Warning: Type annotations on keyword arguments not currently supported in recipes. Type information has been discarded
└ @ RecipesBase ~/.julia/packages/RecipesBase/92zOw/src/RecipesBase.jl:116
┌ Warning: Type annotations on keyword arguments not currently supported in recipes. Type information has been

In [2]:
using DelimitedFiles

Load some packages to make nice figures

In [3]:
using Plots
default(tickfont = font("CMU Serif", 9), 
        titlefont = font("CMU Serif", 14), 
        guidefont = font("CMU Serif", 12),
        legendfont = font("CMU Serif", 10),
        grid = false)
# Plots.font("sans-serif")
# clibrary(:colorbrewer)
# gr()
pyplot()

using LaTeXStrings
# PyPlot.rc("text", usetex = "true")
# rcParams = PyPlot.PyDict(PyPlot.matplotlib."rcParams")
# rcParams["text.usetex"] = true;
PyPlot.rc("font", family = "CMU Serif")
PyPlot.matplotlib[:rc]("mathtext",fontset="cm")        #computer modern font 
PyPlot.matplotlib[:rc]("font",family="serif",size=12)

using ColorSchemes

In [34]:
path = "/media/mat/HDD/AdaptiveTransportMap/notebooks/lorenz96/data/"

β_array = collect(0.95:0.01:1.05)
Ne_array = [60, 100, 200, 400, 600]
L_array = collect(1.0:10.0);

# model, data = setup_lorenz96(path, Ne_array);

Progress: 100%|█████████████████████████████████████████| Time: 0:00:06


Ne 60 RMSE: 2.9534950839158824


Progress: 100%|█████████████████████████████████████████| Time: 0:00:10


Ne 100 RMSE: 1.5828930559308896


Progress: 100%|█████████████████████████████████████████| Time: 0:00:21


Ne 200 RMSE: 0.9688875185932705


Progress: 100%|█████████████████████████████████████████| Time: 0:00:39


Ne 400 RMSE: 0.8145890258184952


Progress: 100%|█████████████████████████████████████████| Time: 0:00:55


Ne 600 RMSE: 0.779881984119389


In [35]:
# save(path*"lorenz96_data.jld", "data", data)

In [36]:
data = load(path*"lorenz96_data.jld", "data")

SyntheticData([0.4, 0.8, 1.2000000000000002, 1.6, 2.0, 2.4000000000000004, 2.8000000000000003, 3.2, 3.6, 4.0  …  2396.4, 2396.8, 2397.2000000000003, 2397.6, 2398.0, 2398.4, 2398.8, 2399.2000000000003, 2399.6, 2400.0], [-0.2974821173632716, 0.501884610567026, -1.1981987411803194, 0.9061672276201467, 0.5505126602205873, 0.3674031762679743, -0.05029338477495031, 1.2141764473990793, -1.5321710427492057, -1.8392124733392539  …  0.501350177777496, 0.20129443164723057, 1.1770483914929308, -0.5314211841940432, -1.0447986057933782, 0.531341375580273, 0.3146441962877211, 0.2563492531890934, -0.25150243881073026, 0.7978032807192892], [2.6349792039764934 3.504181125623047 … -1.7977571208561207 0.09487460971575813; 2.4343783870406344 3.738982713804183 … 2.559562448914188 5.304979080072626; … ; 2.482110373299789 5.116572417455944 … 3.683128109587375 5.795831132353801; 3.176124201010255 5.83946347959046 … 8.230989412210743 0.2453778555195535], [2.156579526522504 3.582613352917209 … -1.910885096436030

In [61]:
Nx = 40
Ny = 20
Δtdyn = 0.01
Δtobs = 0.4

σx = 1e-6
σy = sqrt(0.5)

ϵx = AdditiveInflation(Nx, zeros(Nx), σx)
ϵy = AdditiveInflation(Ny, zeros(Ny), σy)
tspinup = 200.0
Tspinup = 2000
tmetric = 400.0
Tmetric = 4000
t0 = 0.0
tf = 600.0
Tf = 6000

Tburn = 2000
Tstep = 4000
Tspinup = 2000

f = lorenz96!
h(x, t) = x[yidx]
# Create a local version of the observation operator
h(x, idx, t) = x[idx]
F = StateSpace(lorenz96!, h)

model = Model(Nx, Ny, Δtdyn, Δtobs, ϵx, ϵy, MvNormal(zeros(Nx), Matrix(1.0*I, Nx, Nx)), Tburn, Tstep, Tspinup, F);

In [62]:
# # Load file
# Ne = 20
# β = 1.0
# Lrad = 3
# X0 = load(path*"set_up_Ne"*string(Ne)*".jld", "X")

# X = zeros(model.Ny + model.Nx, Ne)
# X[model.Ny+1:model.Ny+model.Nx,:] .= copy(X0)
# J = model.Tstep
# t0 = model.Tspinup*model.Δtobs
# F = model.F

# Δ = 2
# yidx = 1:Δ:model.Nx
# idx = vcat(collect(1:length(yidx))', collect(yidx)')

# enkf = SeqStochEnKF(x->x, F.h, MultiplicativeInflation(β), model.ϵy, model.Ny, model.Nx,
#                     model.Δtdyn, model.Δtobs,
#                     idx, zeros(model.Nx+1, Ne), false, true)

# # Create Localization structure
# Gxy(i,j) = periodicmetric!(i,yidx[j], model.Nx)
# Gyy(i,j) = periodicmetric!(yidx[i],yidx[j], model.Nx)
# Loc = Localization(Lrad, Gxy, Gyy)

# @time statehist = seqassim(F, data, J, model.ϵx, enkf, X, model.Ny, model.Nx, t0, Loc);

Benchmark for the Lorenz-96 problem with the sequential stochastic EnkF

In [63]:
β_array = collect(0.97:0.01:1.05)
Ne_array = [60, 100, 200, 400, 600]
L_array = collect(4:13.0);

In [64]:
metric_enkf = benchmark_lorenz96(model, data, path, Ne_array, β_array, L_array);

(Ne, β, Lrad) = (60, 0.97, 4.0)
 21.675294 seconds (202.57 M allocations: 25.014 GiB, 9.58% gc time)
Ne = 60
Ne 60& β 0.97& Lrad 4.0 RMSE: 1.0741740828090405
(Ne, β, Lrad) = (60, 0.97, 5.0)
 21.028247 seconds (202.22 M allocations: 24.996 GiB, 9.88% gc time)
Ne = 60
Ne 60& β 0.97& Lrad 5.0 RMSE: 1.0234471799795102
(Ne, β, Lrad) = (60, 0.97, 6.0)
 20.907889 seconds (202.22 M allocations: 24.996 GiB, 9.77% gc time)
Ne = 60
Ne 60& β 0.97& Lrad 6.0 RMSE: 1.050136197377068
(Ne, β, Lrad) = (60, 0.97, 7.0)
 21.498876 seconds (202.22 M allocations: 24.996 GiB, 9.79% gc time)
Ne = 60
Ne 60& β 0.97& Lrad 7.0 RMSE: 1.0274211646546674
(Ne, β, Lrad) = (60, 0.97, 8.0)
 21.336839 seconds (202.22 M allocations: 24.996 GiB, 9.81% gc time)
Ne = 60
Ne 60& β 0.97& Lrad 8.0 RMSE: 1.099065435027503
(Ne, β, Lrad) = (60, 0.97, 9.0)
 20.818962 seconds (202.22 M allocations: 24.996 GiB, 9.88% gc time)
Ne = 60
Ne 60& β 0.97& Lrad 9.0 RMSE: 1.121453406705511
(Ne, β, Lrad) = (60, 0.97, 10.0)
 21.020974 seconds (20

 21.430036 seconds (202.22 M allocations: 24.996 GiB, 9.82% gc time)
Ne = 60
Ne 60& β 1.02& Lrad 6.0 RMSE: 0.9942109888408188
(Ne, β, Lrad) = (60, 1.02, 7.0)
 21.565717 seconds (202.22 M allocations: 24.996 GiB, 10.21% gc time)
Ne = 60
Ne 60& β 1.02& Lrad 7.0 RMSE: 1.0026809619933585
(Ne, β, Lrad) = (60, 1.02, 8.0)
 21.544286 seconds (202.22 M allocations: 24.996 GiB, 9.73% gc time)
Ne = 60
Ne 60& β 1.02& Lrad 8.0 RMSE: 1.0709478243875619
(Ne, β, Lrad) = (60, 1.02, 9.0)
 21.894143 seconds (202.22 M allocations: 24.996 GiB, 11.26% gc time)
Ne = 60
Ne 60& β 1.02& Lrad 9.0 RMSE: 1.0633189676647374
(Ne, β, Lrad) = (60, 1.02, 10.0)
 21.119709 seconds (202.22 M allocations: 24.996 GiB, 9.27% gc time)
Ne = 60
Ne 60& β 1.02& Lrad 10.0 RMSE: 1.1674016680791042
(Ne, β, Lrad) = (60, 1.02, 11.0)
 21.254916 seconds (202.22 M allocations: 24.996 GiB, 9.27% gc time)
Ne = 60
Ne 60& β 1.02& Lrad 11.0 RMSE: 1.280447860949633
(Ne, β, Lrad) = (60, 1.02, 12.0)
 21.307122 seconds (202.22 M allocations: 24.9

Ne 100& β 0.98& Lrad 7.0 RMSE: 0.907824068556991
(Ne, β, Lrad) = (100, 0.98, 8.0)
 35.177713 seconds (335.02 M allocations: 39.008 GiB, 9.17% gc time)
Ne = 100
Ne 100& β 0.98& Lrad 8.0 RMSE: 0.9555490045083257
(Ne, β, Lrad) = (100, 0.98, 9.0)
 35.046621 seconds (335.02 M allocations: 39.008 GiB, 9.52% gc time)
Ne = 100
Ne 100& β 0.98& Lrad 9.0 RMSE: 0.9445408841245556
(Ne, β, Lrad) = (100, 0.98, 10.0)
 35.535368 seconds (335.02 M allocations: 39.008 GiB, 9.27% gc time)
Ne = 100
Ne 100& β 0.98& Lrad 10.0 RMSE: 0.9607702545660266
(Ne, β, Lrad) = (100, 0.98, 11.0)
 35.217939 seconds (335.02 M allocations: 39.008 GiB, 9.31% gc time)
Ne = 100
Ne 100& β 0.98& Lrad 11.0 RMSE: 0.9435662028698507
(Ne, β, Lrad) = (100, 0.98, 12.0)
 35.245711 seconds (335.02 M allocations: 39.008 GiB, 9.72% gc time)
Ne = 100
Ne 100& β 0.98& Lrad 12.0 RMSE: 1.0034851813460408
(Ne, β, Lrad) = (100, 0.98, 13.0)
 34.992037 seconds (335.02 M allocations: 39.008 GiB, 8.97% gc time)
Ne = 100
Ne 100& β 0.98& Lrad 13.0 RM

Ne 100& β 1.03& Lrad 8.0 RMSE: 0.9018873511340755
(Ne, β, Lrad) = (100, 1.03, 9.0)
 35.346935 seconds (335.02 M allocations: 39.008 GiB, 9.75% gc time)
Ne = 100
Ne 100& β 1.03& Lrad 9.0 RMSE: 0.9610505229830859
(Ne, β, Lrad) = (100, 1.03, 10.0)
 34.598835 seconds (335.02 M allocations: 39.008 GiB, 9.19% gc time)
Ne = 100
Ne 100& β 1.03& Lrad 10.0 RMSE: 0.9629879384641047
(Ne, β, Lrad) = (100, 1.03, 11.0)
 34.829198 seconds (335.02 M allocations: 39.008 GiB, 9.35% gc time)
Ne = 100
Ne 100& β 1.03& Lrad 11.0 RMSE: 0.9391303098225164
(Ne, β, Lrad) = (100, 1.03, 12.0)
 34.714075 seconds (335.02 M allocations: 39.008 GiB, 9.33% gc time)
Ne = 100
Ne 100& β 1.03& Lrad 12.0 RMSE: 0.999326384133136
(Ne, β, Lrad) = (100, 1.03, 13.0)
 35.304127 seconds (335.02 M allocations: 39.008 GiB, 9.79% gc time)
Ne = 100
Ne 100& β 1.03& Lrad 13.0 RMSE: 1.032034688328475
(Ne, β, Lrad) = (100, 1.04, 4.0)
 34.875464 seconds (335.02 M allocations: 39.008 GiB, 9.35% gc time)
Ne = 100
Ne 100& β 1.04& Lrad 4.0 RMS

Progress:  40%|████████████████▍                        |  ETA: 2:08:32

...

Progress:  80%|████████████████████████████████▊        |  ETA: 2:01:35

Ne 400& β 1.05& Lrad 13.0 RMSE: 0.8062227002063768
(Ne, β, Lrad) = (600, 0.97, 4.0)
212.246330 seconds (2.00 G allocations: 214.186 GiB, 13.22% gc time)
Ne = 600
Ne 600& β 0.97& Lrad 4.0 RMSE: 0.9775616685097214
(Ne, β, Lrad) = (600, 0.97, 5.0)
229.733217 seconds (2.00 G allocations: 214.186 GiB, 12.68% gc time)
Ne = 600
Ne 600& β 0.97& Lrad 5.0 RMSE: 0.9199393966472087
(Ne, β, Lrad) = (600, 0.97, 6.0)
210.768311 seconds (2.00 G allocations: 214.186 GiB, 13.27% gc time)
Ne = 600
Ne 600& β 0.97& Lrad 6.0 RMSE: 0.871373051363523
(Ne, β, Lrad) = (600, 0.97, 7.0)
210.143808 seconds (2.00 G allocations: 214.186 GiB, 13.26% gc time)
Ne = 600
Ne 600& β 0.97& Lrad 7.0 RMSE: 0.8531138789076744
(Ne, β, Lrad) = (600, 0.97, 8.0)
210.860869 seconds (2.00 G allocations: 214.186 GiB, 13.30% gc time)
Ne = 600
Ne 600& β 0.97& Lrad 8.0 RMSE: 0.8218337290058728
(Ne, β, Lrad) = (600, 0.97, 9.0)
210.696648 seconds (2.00 G allocations: 214.186 GiB, 13.28% gc time)
Ne = 600
Ne 600& β 0.97& Lrad 9.0 RMSE: 0.8

Ne 600& β 1.02& Lrad 4.0 RMSE: 0.972034701055285
(Ne, β, Lrad) = (600, 1.02, 5.0)
239.714356 seconds (2.00 G allocations: 214.186 GiB, 12.38% gc time)
Ne = 600
Ne 600& β 1.02& Lrad 5.0 RMSE: 0.9070888497739718
(Ne, β, Lrad) = (600, 1.02, 6.0)
264.209675 seconds (2.00 G allocations: 214.186 GiB, 10.82% gc time)
Ne = 600
Ne 600& β 1.02& Lrad 6.0 RMSE: 0.8671380980240828
(Ne, β, Lrad) = (600, 1.02, 7.0)
260.839761 seconds (2.00 G allocations: 214.186 GiB, 10.87% gc time)
Ne = 600
Ne 600& β 1.02& Lrad 7.0 RMSE: 0.8481698196487931
(Ne, β, Lrad) = (600, 1.02, 8.0)
211.282418 seconds (2.00 G allocations: 214.186 GiB, 13.25% gc time)
Ne = 600
Ne 600& β 1.02& Lrad 8.0 RMSE: 0.8205309908701154
(Ne, β, Lrad) = (600, 1.02, 9.0)
211.083908 seconds (2.00 G allocations: 214.186 GiB, 13.29% gc time)
Ne = 600
Ne 600& β 1.02& Lrad 9.0 RMSE: 0.8080764214291182
(Ne, β, Lrad) = (600, 1.02, 10.0)
210.941003 seconds (2.00 G allocations: 214.186 GiB, 13.27% gc time)
Ne = 600
Ne 600& β 1.02& Lrad 10.0 RMSE: 0.

Progress: 100%|█████████████████████████████████████████| Time: 15:15:42


Ne 600& β 1.05& Lrad 13.0 RMSE: 0.8041609031279165


In [65]:
save(path*"metric_enkf.jld", "metric", metric_enkf)

Benchmark for the Lorenz-96 problem with the sequential stochastic radial map filter

In [66]:
Lrad_array = Float64.(collect(1:10))
nonid_array = Float64.(collect(1:12));

p = 0

In [67]:
p = 0

0

In [ ]:
metric_srmf0 = benchmark_srmf_lorenz96(model, data, path, Ne_array, β_array, Lrad_array, nonid_array, p);

(Ne, β, Lrad, nonid_rad) = (60, 0.97, 1.0, 1.0)


Progress: 100%|█████████████████████████████████████████| Time: 0:01:00


 60.829792 seconds (839.54 M allocations: 75.073 GiB, 12.53% gc time)
Ne = 60
Ne 60 & β 0.97 & Lrad 1.0 & nonid_rad 1.0 & RMSE: 1.2872131666651057
(Ne, β, Lrad, nonid_rad) = (60, 0.97, 1.0, 2.0)


Progress: 100%|█████████████████████████████████████████| Time: 0:01:23


 83.526749 seconds (1.25 G allocations: 105.460 GiB, 11.68% gc time)
Ne = 60
Ne 60 & β 0.97 & Lrad 1.0 & nonid_rad 2.0 & RMSE: 1.2656368039553372
(Ne, β, Lrad, nonid_rad) = (60, 0.97, 1.0, 3.0)


...

Progress: 100%|█████████████████████████████████████████| Time: 0:02:39


159.896628 seconds (2.89 G allocations: 233.748 GiB, 20.74% gc time)
Ne = 60
Ne 60 & β 1.0 & Lrad 7.0 & nonid_rad 5.0 & RMSE: 1.119479183191352
(Ne, β, Lrad, nonid_rad) = (60, 1.0, 7.0, 6.0)


Progress: 100%|█████████████████████████████████████████| Time: 0:03:10


190.748629 seconds (3.47 G allocations: 289.668 GiB, 20.40% gc time)
Ne = 60
Ne 60 & β 1.0 & Lrad 7.0 & nonid_rad 6.0 & RMSE: 1.1477711155795487
(Ne, β, Lrad, nonid_rad) = (60, 1.0, 7.0, 7.0)


Progress:   9%|███▊                                     |  ETA: 0:03:37

In [ ]:
save(path*"metric_srmf"*string(p)*".jld", "metric", metric_srmf0)

p = 1

In [ ]:
p = 1

In [ ]:
metric_srmf1 = benchmark_srmf_lorenz96(model, data, path, Ne_array, β_array, Lrad_array, nonid_array, p);

In [ ]:
save(path*"metric_srmf"*string(p)*".jld", "metric", metric_srmf1)

p = 2

In [ ]:
p = 2

In [ ]:
metric_srmf2 = benchmark_srmf_lorenz96(model, data, path, Ne_array, β_array, Lrad_array, nonid_array, p);

In [ ]:
save(path*"metric_srmf"*string(p)*".jld", "metric", metric_srmf2)

Benchmark for the Lorenz-96 problem with the sequential stochastic adaptive radial map filter

p = 0

In [ ]:
p = 0

In [ ]:
metric_sadaptivermf0 = benchmark_sadaptivermf_lorenz96(model, data, path, Ne_array, β_array, p);

In [ ]:
save(path*"metric_sadaptivermf"*string(p)*".jld", "metric", metric_sadaptivermf0)

p = 1

In [ ]:
p = 1

In [ ]:
metric_sadaptivermf1 = benchmark_sadaptivermf_lorenz96(model, data, path, Ne_array, β_array, p);

In [ ]:
save(path*"metric_sadaptivermf"*string(p)*".jld", "metric", metric_sadaptivermf0)

p = 2

In [ ]:
p = 2

In [ ]:
metric_sadaptivermf2 = benchmark_sadaptivermf_lorenz96(model, data, path, Ne_array, β_array, p);

In [ ]:
save(path*"metric_sadaptivermf"*string(p)*".jld", "metric", metric_sadaptivermf0)